Treinamento e avaliação de modelos para problema de classificação de câncer de mama a partir de imagem digital, em maligno ou benigno. A base foi criada por pesquisadores da Universidade de Wisconsin nos Estados Unidos.

Link para a base no Scikit Learn:
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

Qtde de atributos igual a 31, sendo:
classe (0=maligno ou 1=benigno)
30 atributos contínuos extraídos do núcleo das células em imagem de câncer de mama.

Tipo dos atributos: numéricos
Qtde de instâncias: 569 (212 malignos e 357 benignos)
Protocolo experimental a ser utilizado: validação cruzada com 5 folds.
Indutores a serem avaliados: Árvores de decisão, KNN, Naive Bayes, SVM, MLP e ensembles (Random Forest, Bagging, AdaBoosting, XGBoosting).


In [4]:


# Importações de bibliotecas
from sklearn.datasets import load_diabetes # Para carregar o dataset de diabetes
from sklearn.model_selection import KFold # Para validação cruzada
from sklearn.preprocessing import StandardScaler # Para normalizar os dados
from sklearn.metrics import r2_score, mean_absolute_error # Métricas de regressão (R2 e MAE)

# Regressores (modelos de regressão)
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor # XGBoosting como GradientBoostingRegressor do sklearn
import xgboost as xgb # Importação explícita para xgb.XGBRegressor, caso queira usar diretamente
import numpy as np # Para cálculos numéricos (média, etc.)
import pandas as pd # Para talvez exibir resultados em DataFrame (não usado diretamente na saída final, mas útil)

# ---
# Carregar dados
X, y = load_diabetes(return_X_y=True)

# Normalização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---
# Modelos a serem avaliados (Regressores)
models = {
    "Árvore de Regressão": DecisionTreeRegressor(random_state=42),
    "KNN": KNeighborsRegressor(),
    "SVR": SVR(),
    "MLP": MLPRegressor(random_state=42, max_iter=1000), # Aumentado max_iter para auxiliar na convergência
    "Random Forest": RandomForestRegressor(random_state=42),
    "Bagging": BaggingRegressor(random_state=42),
    "XGBoosting": GradientBoostingRegressor(random_state=42) # Usando GradientBoostingRegressor do sklearn
    # Se quiser usar a biblioteca 'xgboost' diretamente, descomente a linha abaixo:
    # "XGBoosting (lib)": xgb.XGBRegressor(random_state=42, eval_metric='mae') # eval_metric para regressão
}

# ---
# Função para treinar o modelo, calcular métricas e retornar resultados usando KFold
def evaluate_model_cv(model, X_data, y_data, n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    r2_scores = []
    mae_scores = []

    # Importa clone aqui para garantir que está disponível na função
    from sklearn.base import clone

    for train_index, test_index in kf.split(X_data):
        X_train, X_test = X_data[train_index], X_data[test_index]
        y_train, y_test = y_data[train_index], y_data[test_index]

        # Clona o modelo para garantir que cada fold comece com um modelo 'novo'
        cloned_model = clone(model)

        cloned_model.fit(X_train, y_train)
        y_pred = cloned_model.predict(X_test)

        r2_scores.append(r2_score(y_test, y_pred))
        mae_scores.append(mean_absolute_error(y_test, y_pred))

    # Retorna a média das métricas de todos os folds
    avg_r2 = np.mean(r2_scores)
    avg_mae = np.mean(mae_scores)

    return avg_r2, avg_mae, model.get_params() # Retorna os parâmetros do modelo original

# ---
# Avaliar todos os modelos
best_model_name = ""
best_r2 = -np.inf # Queremos maximizar R2, então começamos com um valor negativo infinito
best_mae = np.inf # Queremos minimizar MAE, então começamos com um valor positivo infinito
best_params = None

all_results = [] # Para armazenar todos os resultados para a tabela final

for model_name, model in models.items():
    avg_r2, avg_mae, params = evaluate_model_cv(model, X_scaled, y)

    # Adiciona os resultados à lista de todos os resultados
    all_results.append({
        'Indutor': model_name,
        'Coeficiente de Determinação (R2)': avg_r2,
        'Erro Médio Absoluto (MAE)': avg_mae,
        'Parâmetros': params
    })

    # Verifica se este é o melhor modelo até agora (baseado no R2)
    if avg_r2 > best_r2:
        best_r2 = avg_r2
        best_mae = avg_mae
        best_model_name = model_name
        best_params = params

# ---
# Exibir a Tabela de Resultados completa
print("---")
print("## Tabela de Resultados (Todos os Regressores)")
print("")
# Cabeçalho da tabela
print("| Indutor | Coeficiente de Determinação (R2) | Erro Médio Absoluto (MAE) |")
print("|:--------------------|:------------------------------------|:--------------------------|")

# Linhas da tabela
for result in all_results:
    # Destaca o melhor modelo
    if result['Indutor'] == best_model_name:
        print(f"| **{result['Indutor']}** | **{result['Coeficiente de Determinação (R2)']:.4f}** | **{result['Erro Médio Absoluto (MAE)']:.4f}** |")
    else:
        print(f"| {result['Indutor']} | {result['Coeficiente de Determinação (R2)']:.4f} | {result['Erro Médio Absoluto (MAE)']:.4f} |")

# ---
# Apresentar as informações do melhor modelo (B.1)
print("\n---")
print("## B.1) Parâmetros do Melhor Modelo")
print(f"\nO melhor regressor encontrado foi o **{best_model_name}**.")
print("Os parâmetros utilizados no treinamento do modelo foram:")
for param, value in best_params.items():
    print(f"- {param}: {value}")

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perce

---
## Tabela de Resultados (Todos os Regressores)

| Indutor | Coeficiente de Determinação (R2) | Erro Médio Absoluto (MAE) |
|:--------------------|:------------------------------------|:--------------------------|
| Árvore de Regressão | -0.1343 | 63.5188 |
| KNN | 0.3903 | 46.7387 |
| SVR | 0.1495 | 58.8854 |
| **MLP** | **0.4497** | **45.1405** |
| Random Forest | 0.4299 | 46.6673 |
| Bagging | 0.3746 | 48.7749 |
| XGBoosting | 0.4164 | 47.3211 |

---
## B.1) Parâmetros do Melhor Modelo

O melhor regressor encontrado foi o **MLP**.
Os parâmetros utilizados no treinamento do modelo foram:
- activation: relu
- alpha: 0.0001
- batch_size: auto
- beta_1: 0.9
- beta_2: 0.999
- early_stopping: False
- epsilon: 1e-08
- hidden_layer_sizes: (100,)
- learning_rate: constant
- learning_rate_init: 0.001
- max_fun: 15000
- max_iter: 1000
- momentum: 0.9
- n_iter_no_change: 10
- nesterovs_momentum: True
- power_t: 0.5
- random_state: 42
- shuffle: True
- solver: adam
- tol: 0.0001
- validation_